In [1]:
from transformers import pipeline
from collections import Counter
from tqdm import tqdm
import pandas as pd
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

In [77]:
# define a custom label set
candidate_labels = [
'Self improvement',
'Relationships',
'Mental Health',
'Social Issues',
'Entertainment',
'Environment',
'Technology',
'Lifestyle',
'Education',
'Politics',
'Religion',
'Culture',
'Economy',
'History',
'Society',
'Health',
'Research',
'Career',
'Travel',
'Sports',
'Music',
'Humor',
'Food',
'Art',]

In [ ]:
# read in the data
data = pd.read_csv('/home/chel/folder/RedDash/data/topic_info_cleaned.csv')
data = data.dropna(subset=['LLM'])

In [78]:
batch_size = 32 # number of sequences to process at once
sequences = data['LLM'].to_list()
results = []

In [79]:
for i in tqdm(range(0, len(sequences), batch_size)):
    results += pipe(sequences[i:i+batch_size], candidate_labels)

100%|██████████| 20/20 [48:24<00:00, 145.21s/it]


In [80]:
# extract the top label for each sequence
li = [lab['labels'][0] for lab in results]
data['Category'] = li

In [94]:
# count the number of times each label appears
Counter(li)

Counter({'Lifestyle': 486,
         'Culture': 446,
         'Self improvement': 386,
         'Entertainment': 373,
         'Humor': 345,
         'Research': 325,
         'Relationships': 256,
         'Technology': 226,
         'Environment': 216,
         'Economy': 199,
         'Career': 180,
         'Travel': 180,
         'History': 175,
         'Health': 172,
         'Food': 167,
         'Society': 156,
         'Religion': 121,
         'Sports': 109,
         'Social Issues': 103,
         'Education': 68,
         'Mental Health': 68,
         'Politics': 57,
         'Music': 52,
         'Art': 38})

In [93]:
data.to_csv('../data/results/topic_info_categories.csv')